In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pprint

from qutip import *
import os

# Matplotlib Einstellungen gemäß den LaTeX-Caption-Formatierungen
plt.rcParams.update({
    #    'text.usetex': True,              # Enable LaTeX for text rendering
    #    'font.family': 'serif',           # Use a serif font family
    #    'font.serif': 'Palatino',         # Set Palatino as the serif font
    #    'text.latex.preamble': r'\usepackage{amsmath}',
    #    'font.size': 20,                   # Font size for general text
    #    'axes.titlesize': 20,              # Font size for axis titles
    #    'axes.labelsize': 20,              # Font size for axis labels
    #    'xtick.labelsize': 20,             # Font size for x-axis tick labels
    #    'ytick.labelsize': 20,             # Font size for y-axis tick labels
    #    'legend.fontsize': 20,             # Font size for legends
    #    'figure.figsize': [8, 6],          # Size of the plot (width x height)
    #    'figure.autolayout': True,         # Automatic layout adjustment
    #    'savefig.format': 'svg',           # Default format for saving figures
    #    'figure.facecolor': 'none',        # Make the figure face color transparent
    #    'axes.facecolor': 'none',          # Make the axes face color transparent
    #    'savefig.transparent': True        # Save figures with transparent background
})
output_dir = r"C:\Users\leopo\OneDrive - UT Cloud\Uni\Semester_9\Master_thesis\Figures_From_Python"
os.makedirs(output_dir, exist_ok=True)

# allows for interactive plots
#%matplotlib notebook

$$
H = H_0 + H_I
$$
$$
H_0 = \hbar  \omega_a \sum_
{i = 1} ^ {N_
{\text
{atoms}}} \sigma_ + ^ {(i)} \sigma_ - ^ {(i)}
+ \hbar \sum_
{i, j = 1} ^ {N_\text
{{atoms}}} J_
{i
j} \sigma_ + ^ {(i)} \sigma_ - ^ {(j)}
$$

$$
H_I = \hbar \Sigma_x
E(t).
$$

$$
\displaystyle
J_
{i
j} = \frac
{\alpha}{ | r_i - r_j |³}.
$$

In [ ]:
# Set the system parameters
n_rings = 1
n_chains = 1  # number of chains
N_atoms = n_chains * n_rings  # number of atoms

# ENERGY LANDSCAPE, c = 1, hbar = 1
fixed_lam = 1  #2 * np.pi / omega_a                # propto omega_a;  energysplitting of the atom, when ground state is set to 0
omega_a = 2 * np.pi / fixed_lam  #1.
alpha = 1.  # coupling strength of the dipoles       Fine structure const?
# LINBLAD OPS
gamma_0 = .1  # decay rate of the atoms
gamma_phi = .1  # dephasing rate of the atoms

# TOPOLOGY
distance = 1 * fixed_lam  # defining topology

# TIME EVOLUTION
last_pulse = 5  # * gamma_0
last_det_t = 2 * last_pulse
time_steps = 100
times = np.linspace(0, last_det_t, time_steps)  # list of times


def count_decimal_digits(number):
    # Convert the number to string
    str_number = str(number)

    # Split the string at the decimal point and count the digits after it
    if '.' in str_number:
        return len(str_number.split('.')[1])
    else:
        return 0  # No digits after decimal if it's an integer


tolerance = count_decimal_digits(times[1]) - 2


########################################               define the geometry                 #############################################
def chain_positions(distance, N_atoms):
    Pos = np.zeros((N_atoms, 3))
    for i in range(N_atoms):
        Pos[i, 2] = i * distance
    return Pos


def z_rotation(angle):
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]])


def ring_positions(distance, n_chains):
    Pos = np.zeros((n_chains, 3))
    dphi = 2 * np.pi / n_chains
    if n_chains == 1:
        radius = 0
    else:
        radius = distance / 2 / np.sin(np.pi / n_chains)
    helper = np.array([radius, 0, 0])
    for i in range(n_chains):
        rotation_matrix = z_rotation(dphi * i)
        Pos[i] = np.matmul(rotation_matrix, helper)
    return Pos


def cyl_positions(distance, N_atoms, n_chains):
    Pos = np.zeros((N_atoms, 3))
    Pos_chain = chain_positions(distance, N_atoms // n_chains)
    Pos_ring = ring_positions(distance, n_chains)
    for i in range(n_chains):
        Pos[i * (N_atoms // n_chains): (i + 1) * (N_atoms // n_chains)] = Pos_chain + Pos_ring[i]
    return Pos


Pos = cyl_positions(distance, N_atoms, n_chains)

# Plotting
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
# Plot the positions
ax.scatter(Pos[:, 0], Pos[:, 1], Pos[:, 2], c='b', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Positions and Dipoles')
ax.axis('equal')
plt.show()
# Define the ground & the excited states
# atomic dofs
atom_g = basis(N_atoms + 1, 0)
atom_es = [basis(N_atoms + 1, i) for i in range(1, N_atoms + 1)]

# combined dofs
sm_list = []  # lowering operators of atomic system
S_x = 0  # collective sigma_x operator for the system

for i in range(N_atoms):
    op = atom_g * atom_es[i].dag()
    sm_list.append(op)
    S_x += op + op.dag()


def Hamilton0(distance, n_rings, n_chains):
    N_atoms = n_chains * n_rings
    Pos = cyl_positions(distance, N_atoms, n_chains)
    H = 0
    for a in range(N_atoms):
        for b in range(N_atoms):
            op = sm_list[a].dag() * sm_list[b]
            if a != b:
                ra, rb = Pos[a, :], Pos[b, :]
                H += alpha / (np.linalg.norm(rb - ra)) ** 3 * op
            else:
                H += omega_a * op  # Diagonals except for |0><0|
    return H


H0 = Hamilton0(distance, n_rings, n_chains)

H0
# Jump / Expect Operators
# Define the decay collapse and dephasing operator for each spin

op_list = sm_list  # Operators combining the cavity (qeye(2)) and the atomic lowering operator (sm)

# Collapse operators
c_op2 = [np.sqrt(gamma_0) * op for op in op_list]  # Individual atom decays
c_op4 = [np.sqrt(gamma_phi) * commutator(op.dag(), op) for op in op_list]  # Individual atom dephasing
c_op_list = c_op2 + c_op4  # Combine all collapse operators

# Expectation operators for measuring populations across atomic ground and excited levels
e_op_list = [
    basis(N_atoms + 1, i) * basis(N_atoms + 1, i).dag()
    for i in range(N_atoms + 1)]
# create the time dependant evolution
k_vec = 2 * np.pi / fixed_lam * np.array([0, 0, 1])

I = S_x
E12 = 1  # Amplitude of laser pulses 1,2
E0s = [E12, E12, E12 / 10]  # Probe pulse is smaller
omegas = [omega_a, omega_a, omega_a]
Delta_ts = [last_pulse / 100, last_pulse / 100, last_pulse / 100]  # narrow width of the pulses
phi12 = 2 * np.pi  # phis = [phi12, phi12, 0]  # Phase-kick?, pulse 1 and 2 are phase locked!


def heaviside(x):
    return 1 if x >= 0 else 0


def El_field(t, args):
    t0 = args['time']
    Delta = args['Delta']
    E = np.cos(np.pi * (t - t0) / (2 * Delta)) ** 2
    E *= heaviside(t - (t0 - Delta)) * heaviside(t0 + Delta - t) * np.cos((args['omega'] * t) + args['phi'])

    return args['E0'] * E


HI = [-I, El_field]  # interaction Hamiltonian with function-based time dependence
H = [H0, HI]

options = Options(store_states=True)
# VIZUALIZE THE FIELD AS AN EXAMPLE
#
t_values = np.linspace(0, 2 * last_pulse, 1000)
args_0 = {
    'phi': phi12,  # Use the phase 0
    'time': Delta_ts[0],  # t0 value = Delta value such that the pulse immediatley starts
    'omega': omegas[0],  # omega value
    'Delta': Delta_ts[0],  # The width should be
    'E0': E0s[0]  # E0 value
}
args_1 = {
    'phi': phi12,
    'time': 5 + Delta_ts[1],
    'omega': omegas[1],
    'Delta': Delta_ts[1],
    'E0': E0s[1]
}
args_2 = {
    'phi': 0,  #last pulse has no phase kick
    'time': 8 + Delta_ts[2],  # Duration for the 2nd pulse
    'omega': omegas[2],  # Omega for the 2nd pulse
    'Delta': Delta_ts[2],  # Delta for the 2nd pulse
    'E0': E0s[2]  # E0 for the 2nd pulse
}
electric_field = [El_field(t, args_0) + El_field(t, args_1) + El_field(t, args_2) for t in t_values]  #
plt.figure()
plt.plot(t_values, electric_field, label='Electric Field E(t)', color='blue')
plt.xlabel('Time (t)')
plt.ylabel('Field Strength (E)')
plt.show()
# evolution with t
# initial state
psiini = basis(N_atoms + 1, 0)  # = |g>_atom

#
# PREPROCESSING
#
# Define the phases, coherence times, and waiting times
phases = [i * np.pi / 2 for i in range(4)]

# Initialize the main dictionary to hold the structure
data_dict_stage1 = {}
data_dict_stage2 = {}

# Loop through each phase φ
for phi in phases:
    data_dict_stage1[phi] = {}  # Stage 1 copy of the phase
    data_dict_stage2[phi] = {}  # Stage 2 copy of the phase
#
# CALCULATIONS
#

# (first laser pulse)
times_0 = times

# Iterate over the phi values
for phi in data_dict_stage1:
    args_0 = {
        'phi': phi12,  # Use the phase 0
        'time': Delta_ts[0],  # t0 value = Delta value such that the pulse immediatley starts
        'omega': omegas[0],  # omega value
        'Delta': Delta_ts[0],  # The width should be
        'E0': E0s[0]  # E0 value
    }
    # Solve the system using mesolve (first laser pulse)
    result_0 = mesolve(H, psiini, times_0, c_ops=c_op_list, e_ops=e_op_list, args=args_0, options=options)

    current_dict = {}

    for i in range(len(times_0)):  # save only the states that make sense
        if (2 * Delta_ts[0] <= times_0[i] <= last_pulse - 2 * (Delta_ts[1] + Delta_ts[2])):
            current_dict[i] = result_0.states[i]  # tau is indirectly stored in the index i (key of the dictionary)

    if current_dict:  # Check if the dictionary is not empty
        data_dict_stage1[phi] = current_dict

#
# PLOT TO CHECK the evolution
#

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
# Plot 1: Electric field vs time
ax1.plot(times_0, [El_field(t, args_0) for t in times_0])
ax1.set_xlabel("Time (arb. units)")
ax1.set_ylabel("Electric Field")
ax1.set_title("Electric Field vs Time")

for i, e_op_expect in enumerate(result_0.expect):
    if i == 0:
        label = r"$|g>_{at}$"
    else:
        label = f"$|e>_{i}$"
    ax2.plot(result_0.times, e_op_expect, label=label)

ax2.set_xlabel("Time (arb. units)")
ax2.set_ylabel("Expectation Value")
ax2.set_title("Evolution of Observables (e_ops) Over Time")
ax2.legend()
plt.show()
#pprint.pprint(data_dict_stage1)